In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.5 MB/s eta 0:00:00


In [3]:
import json
import random
import pandas as pd
import csv
import os

## Prep dataset

In [4]:
df = pd.read_json("hf://datasets/FreedomIntelligence/medical-o1-verifiable-problem/medical_o1_verifiable_problem.json")
df.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


In [5]:
df.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


## Set up model definition

In [6]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the Huatuo model
model_id = "FreedomIntelligence/HuatuoGPT-o1-8B"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)


In [7]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [8]:
tokenizer.get_chat_template()

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

## Eval of Huatuo

In [9]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('HF_TOKEN'))


config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
x = df.loc[0, "Open-ended Verifiable Question"]
y_star = df.loc[0, "Ground-True Answer"]

In [11]:
print('x:', x)
print('y_star:', y_star)

x: An 88-year-old woman with osteoarthritis is experiencing mild epigastric discomfort and has vomited material resembling coffee grounds multiple times. Considering her use of naproxen, what is the most likely cause of her gastrointestinal blood loss?
y_star: Gastric ulcer


In [12]:
init_prompt = f"""<question>
{x}
</question>
Please respond to the above question <question> using the Chain of Thought (CoT) reasoning method.
Your response should consist of multiple steps, each of which includes three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
- **’Inner Thinking’**: This is the step where thinking is done. Note that multiple ’Inner Thinking’ steps are
required to describe thorough reasoning. Each step should first generate a brief title.
- **’Final Conclusion’**: At this stage, you summarize the correct reasoning from previous ’Inner Thinking’
steps and provide the final answer. No title is required here.
- **’Verification’**: At this stage, you verify the conclusion from the "Final Conclusion" step. If the
conclusion holds, end the process. If not, return to "Inner Thinking" for further reasoning. No title is required
here.
The output format must strictly follow the JSON structure
"""

In [13]:
outputs = pipe(
        init_prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
response = outputs[0]["generated_text"]
print(response)

```json
[
    {
        "action": "Inner Thinking",
        "text": "Let's start by thinking about what's going on with the patient. She's an 88-year-old woman who has osteoarthritis, so she probably takes medications like naproxen to help with pain. Now, she's been having some mild epigastric discomfort and has even thrown up something like coffee grounds several times. That's concerning because it sounds a lot like she's vomiting blood."
    },
    {
        "action": "Inner Thinking",
        "text": "Alright, let's consider what could be causing this vomiting of blood. It often means there’s some kind of bleeding in the stomach or esophagus, especially if someone has been taking NSAIDs like naproxen over time. So, I'm wondering if these medications could be wearing away at her stomach lining and leading to ulcers."
    },
    {
        "action": "Inner Thinking",
        "text": "So, what exactly does naproxen do? Well, it's known for reducing inflammation but can also irritate the

In [14]:
verif_prompt = f"""<Model Response>
{response}
</Model Response>
<Reference Answer>
{y_star}
</Reference Answer>
You are provided with a model-generated response (<Model Response>) and a reference answer (<Reference
Answer>). Compare the model response with the reference answer and determine its correctness. Your task
is to simply output "True" if the response is correct, and "False" otherwise"""

In [15]:
outputs = pipe(
        verif_prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
response = outputs[0]["generated_text"]
print(response)

.

## Thinking

Okay, let's break down what's happening here. We've got an elderly woman experiencing some worrying symptoms. She's been vomiting something that looks like coffee grounds, and she has mild epigastric discomfort. This coffee-ground stuff usually means there's some bleeding in the stomach or esophagus.

Now, what stands out is she's taking naproxen, which is an NSAID. These meds are great for pain relief, but they're notorious for causing trouble in the stomach. Prolonged use can lead to gastric mucosal damage, which, unfortunately, can result in peptic ulcers. These ulcers can bleed, and that could explain why she's vomiting blood-like material.

It makes sense considering she's got osteoarthritis and would likely be using naproxen to manage her chronic joint pain. The longer you take naproxen, the higher the risk of stomach issues.

So, putting all of this together, the combination of her symptoms and medication history strongly points towards gastric ulcer as the under

### Defining Prompt Templates

In [16]:
init_prompt = """<question>
{x}
</question>
Please respond to the above question <question> using the Chain of Thought (CoT) reasoning method.
Your response should consist of multiple steps, each of which includes three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
- **’Inner Thinking’**: This is the step where thinking is done. Note that multiple ’Inner Thinking’ steps are
required to describe thorough reasoning. Each step should first generate a brief title.
- **’Final Conclusion’**: At this stage, you summarize the correct reasoning from previous ’Inner Thinking’
steps and provide the final answer. No title is required here.
- **’Verification’**: At this stage, you verify the conclusion from the "Final Conclusion" step. If the
conclusion holds, end the process. If not, return to "Inner Thinking" for further reasoning. No title is required
here.
The output format must strictly follow the JSON structure
"""

In [17]:
verif_prompt = """<Model Response>
{response}
</Model Response>
<Reference Answer>
{y_star}
</Reference Answer>
You are provided with a model-generated response (<Model Response>) and a reference answer (<Reference
Answer>). Compare the model response with the reference answer and determine its correctness. Your task
is to simply output "True" if the response is correct, and "False" otherwise"""

Exploring New Paths

In [18]:
exploring_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section.
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning by exploring new approaches to solving this problem and construct a new Final
Conclusion.
"""

Backtracking

In [19]:
backtracking_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section.
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning using **backtracking** to revisit earlier points of reasoning and construct a new Final
Conclusion.
"""

Verifications

In [20]:
verif_search_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section.
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning by making precise **corrections** to address prior flaws and construct a new Final
Conclusion.
"""

Corrections

In [21]:
corr_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning by conducting a thorough **validation** process to ensure validity and construct a
new Final Conclusion.
"""

## Function Form

In [22]:
def clean_json_output(raw_text):
    return raw_text.strip().removeprefix("```json").removeprefix("```").removesuffix("```").strip()

In [23]:
def run_huatuo(prompt):

  outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )

  response = outputs[0]["generated_text"]

  return response

In [24]:
def dsft_approach(x, y_star):

  prompts = []

  prompt = init_prompt.format(x=x)
  prompts.append(prompt)
  response = run_huatuo(prompt)

  v_prompt = verif_prompt.format(response=response, y_star=y_star)
  verif_result = run_huatuo(v_prompt)

  clean_res = False

  if 'True' in verif_result or 'true' in verif_result:
    clean_res = True

  if clean_res == False:

    result = False
    idx = 0

    searching_prompts = [exploring_prompt, backtracking_prompt, verif_search_prompt, corr_prompt]
    random.shuffle(searching_prompts)

    searching_prompts = searching_prompts[:3]

    while result == False and idx <= 2:

      prompt = searching_prompts[idx].format(x=x, response=response)
      prompts.append(prompt)
      response = run_huatuo(prompt)

      v_prompt = verif_prompt.format(response=response, y_star=y_star)
      verif_result = run_huatuo(v_prompt)

      if 'True' in verif_result or 'true' in verif_result:
        clean_res = True
        result = True
      else:
        idx += 1

  return response, prompts, clean_res

In [25]:
results = []

csv_path = '/content/drive/MyDrive/MIDS/dsft_huatuo_results_56.csv'
fieldnames = ['Answer', 'Prompts', 'post_ft_results']

if not os.path.exists(csv_path):
    with open(csv_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

for i in range(56, 300):

  print(i)

  x = df.loc[i, "Open-ended Verifiable Question"]
  y_star = df.loc[i, "Ground-True Answer"]

  answer, prompts, res = dsft_approach(x, y_star)

  with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([answer, prompts, res])

  results.append({
        "Answer": answer,
        "Prompts": prompts,
        "post_ft_results": res
    })

56
57
58


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299


In [ ]:
# results_df = pd.DataFrame(results)

# results_df['num_prompts'] = results_df['Prompts'].apply(len)
# results_df["baseline_results"] = (results_df['num_prompts'] == 1).astype(int)
# results_df['post_ft_results'] = results_df['post_ft_results'].astype(int)

# results_df.head(10)